05-xrefs
Get xrefs from a variety of sources
- Drugs: 
UMLS has mesh xrefs. From mesh, we can get UNII and CAS. From UNII_FDA, we can get inchikeys 
(lookup using cas or unii). From chembl, we can get chembl IDs from the inchikeys
So: UMLS -> mesh -> unii/cas -> inchikey -> chembl
insane, I know.
- Anatomy: uberon has umls xrefs
- disease: DO has umls, umls has NCI, ICD10PCS, SNOMEDCT_US, ICD10CM, OMIM
- proteins: umls has uniprot xrefs
- biological_process_or_activity/activity_and_behavior: umls has GO
- gene: umls has HGNC and OMIM

In [1]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
import shelve
import re
from collections import defaultdict, Counter
from tqdm import tqdm, tqdm_notebook
from itertools import chain
from more_itertools import chunked
from collections import Counter
from pprint import pprint
import requests
from pyquery import PyQuery as pq

In [2]:
uri_to_curie = lambda s: s.split("/")[-1].replace("_", ":")

In [3]:
# edges = pd.read_csv('edges_biolink.csv')
nodes = pd.read_csv("nodes_biolink.csv", index_col=0)

In [4]:
nodes.head()

,LABEL,umls_type,umls_type_label,blm_type
ID,,,,
C0061133,gastrin releasing peptide (14-27),T116,"Amino Acid, Peptide, or Protein",protein
C1523610,"regulation of tube length, open tracheal system",T042,Organ or Tissue Function,biological_process_or_activity
C0312636,Antibody to hepatitis E virus,T116|T129,"Amino Acid, Peptide, or Protein|Immunologic Fa...",biological_entity
C0539817,cytochrome p30,T116|T126,"Amino Acid, Peptide, or Protein|Enzyme",protein
C0406240,Photosensitive atopic dermatitis,T047,Disease or Syndrome,disease_or_phenotypic_feature


In [5]:
nodes.blm_type.value_counts()

chemical_substance                58812
disease_or_phenotypic_feature     36248
gene                              20695
biological_entity                 14905
protein                           12644
gross_anatomical_structure         8472
biological_process_or_activity     6887
anatomical_entity                  2750
cell_component                     1644
cell                               1099
activity_and_behavior               935
phenotypic_feature                  393
genomic_entity                      174
Name: blm_type, dtype: int64

In [6]:
## parse UMLS flat file to get all UMLS xrefs
# see: https://www.ncbi.nlm.nih.gov/books/NBK9685/

In [7]:
names = "CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X".split(",")
umls = pd.read_csv("MRCONSO_ENG.RRF.gz", delimiter="|", names=names, index_col=None)
# only get CUIs in our list of nodes
umls = umls[umls.CUI.isin(nodes.index)]

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
umls['xref'] = umls.SAB + ":" + umls.CODE.map(str)
# fix this MSH MESH nonsense
umls.xref = umls.xref.str.replace("MSH:", "MESH:")
# NCI_FDA is UNII
umls.xref = umls.xref.str.replace("NCI_FDA:", "UNII:")

In [9]:
umls.head(2)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X,xref
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,RXNORM:1926948
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,MTH:NOCODE


In [10]:
XREF = dict(umls.groupby("CUI")['xref'].apply(set))
XREF = defaultdict(set, XREF)
print(XREF['C0591520'])

{'RXNORM:151775', 'RCD:x02cs', 'CHV:0000041179', 'MESH:D000068298'}


In [12]:
# all xrefs we get from umls
# Counter(list(chain(*[list(map(lambda x:x.split(":",1)[0], y)) for y in XREF.values()])))

### Chemicals and drugs

In [13]:
# what xrefs are on chemicals?
chem_umls = nodes[nodes.blm_type == "chemical_substance"].index
xref_chem = {k:v for k,v in XREF.items() if k in chem_umls}
print(len(chem_umls))
c = Counter(list(chain(*[list(map(lambda x:x.split(":",1)[0], y)) for y in xref_chem.values()])))
pprint(c.most_common(25))
# nearly all have a mesh ID. not much of anything else
# neither mesh nor umls have inchikeys, or inchi, or smiles or anything usefull for linking out
# blech

58812
[('MESH', 51017),
 ('SNOMEDCT_US', 14733),
 ('NCI', 10568),
 ('RXNORM', 9325),
 ('NDFRT', 9165),
 ('CHV', 8270),
 ('MMSL', 7835),
 ('UNII', 6264),
 ('NDDF', 6209),
 ('MTHSPL', 6006),
 ('RCD', 5985),
 ('SNMI', 5369),
 ('MTH', 5339),
 ('LNC', 5261),
 ('VANDF', 3962),
 ('DRUGBANK', 3284),
 ('ATC', 3113),
 ('CSP', 2891),
 ('MEDCIN', 2817),
 ('SNM', 2665),
 ('PDQ', 2288),
 ('AOD', 1932),
 ('LCH_NW', 1809),
 ('NCI_NCI-GLOSS', 1501),
 ('USPMG', 1205)]


In [14]:
import pandas as pd
import requests
pd.set_option("display.width", 120)
import sys, os
sys.path.insert(0, "/home/gstupp/projects/WikidataIntegrator")
from wikidataintegrator import wdi_helpers, wdi_core, wdi_login

URL = "http://id.nlm.nih.gov/mesh/sparql"
PREFIX = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
"""

def sparql_query(query):
    params = {'query': PREFIX + query, 'format': 'JSON', 'limit': 1000, 'offset': 0}
    r = requests.get(URL, params=params)
    res = [{k: v['value'] for k, v in x.items()} for x in r.json()['results']['bindings']]
    t = tqdm()
    while True:
        t.update(1)
        params['offset'] += 1000
        r = requests.get(URL, params=params).json()['results']['bindings']
        if not r:
            break
        res.extend([{k: v['value'] for k, v in x.items()} for x in r])
    df = pd.DataFrame(res)
    return df

In [15]:
query = """
SELECT distinct ?mesh ?meshLabel ?r ?rr
FROM <http://id.nlm.nih.gov/mesh> WHERE {
  ?mesh meshv:active 1 .
  ?mesh meshv:preferredMappedTo ?p .
  ?p meshv:treeNumber ?treeNum .
  FILTER(STRSTARTS(STR(?treeNum), "http://id.nlm.nih.gov/mesh/D")) .
  ?mesh rdfs:label ?meshLabel .
  ?mesh meshv:preferredConcept [meshv:registryNumber ?r] .
  #OPTIONAL {?mesh meshv:preferredConcept [meshv:relatedRegistryNumber ?rr]}
}
"""
df = sparql_query(query)

214it [06:12,  2.48s/it]

In [16]:
df.r = df.r.replace("0", pd.np.NaN)
df.dropna(subset=["r"], inplace=True)
df = df[~df.r.str.startswith("EC ")]
df.mesh = df.mesh.str.replace("http://id.nlm.nih.gov/mesh/", "")
df.set_index("mesh", inplace=True)

In [17]:
df.to_csv("mesh_xrefs.csv")
df.head()

,meshLabel,r
mesh,,
C016271,bromine chloride,7G62XY5724
C016769,Syn-ergel,66799-40-4
C017138,CMB-dextran,37307-31-6
C017827,bacterio-opsin,54577-62-7
C017911,2-(methylamino)isobutyric acid,2566-34-9


In [18]:
mesh_xrefs = pd.read_csv("mesh_xrefs.csv", index_col=0)
mesh_xrefs.r = mesh_xrefs.r.apply(lambda x: "CAS:" + x if "-" in x else "UNII:" + x)
mesh_xrefs = mesh_xrefs.groupby("mesh").r.apply(set).to_dict()
mesh_xrefs = {"MESH:"+k:v for k,v in mesh_xrefs.items()}
len(mesh_xrefs)

37158

In [19]:
for k,v in xref_chem.items():
    for vv in list(v):
        if vv in mesh_xrefs:
            v.update(mesh_xrefs[vv])

In [20]:
# download: 'http://fdasis.nlm.nih.gov/srs/download/srs/UNII_Data.zip'
unii_df = pd.read_csv("UNII Records 20Jun2018.txt", dtype=str, sep='\t', low_memory=False)
unii_df.dropna(subset=['INCHIKEY'], inplace=True)

In [21]:
unii_df.head()

,UNII,PT,RN,EC,NCIT,RXCUI,PUBCHEM,ITIS,NCBI,PLANTS,GRIN,MPNS,INN_ID,MF,INCHIKEY,SMILES,INGREDIENT_TYPE
0,0001H6R5H1,CEROUS SALICYLATE,526-17-0,NaN,NaN,NaN,76966289,NaN,NaN,NaN,NaN,NaN,NaN,3C7H5O3.Ce,RBJPAJHTYHKKTB-UHFFFAOYSA-K,[Ce+3].OC1=C(C=CC=C1)C([O-])=O.OC2=C(C=CC=C2)C...,INGREDIENT SUBSTANCE
1,000360VJE1,DI(DEHYDROABIETYL)AMINE ACETATE,53404-27-6,NaN,NaN,NaN,76969106,NaN,NaN,NaN,NaN,NaN,NaN,C40H59N.C2H4O2,SETIUTJHVNMKFM-TUICDNFPSA-N,CC(O)=O.CC(C)C1=CC=C2C(CC[C@H]3[C@](C)(CNC[C@]...,INGREDIENT SUBSTANCE
4,00072J7XWS,GERMANIUM,7440-56-4,231-164-3,C95170,4784,6326954,NaN,NaN,NaN,NaN,NaN,NaN,Ge,QUZPNFFHZPRKJD-UHFFFAOYSA-N,[Ge],INGREDIENT SUBSTANCE
7,0009YL8Y42,"5-(3-BROMO-4,5-DIMETHOXYBENZYL)PYRIMIDINE-2,4-...",NaN,NaN,NaN,NaN,76963562,NaN,NaN,NaN,NaN,NaN,NaN,C13H15BrN4O3,WXKLFFZKCBICOL-UHFFFAOYSA-N,COC1=C(OC)C(OBr)=CC(CC2=C(N)N=C(N)N=C2)=C1,INGREDIENT SUBSTANCE
9,000F949089,SPIROFYLLINE,98204-48-9,NaN,C74214,NaN,3086451,NaN,NaN,NaN,NaN,NaN,6195,C24H28N6O5,DSRGPEAMMDAUGF-UHFFFAOYSA-N,CN1C2=C(N(CC(=O)N3CC4(CCN(CCC5=CC=CC=C5)CC4)OC...,INGREDIENT SUBSTANCE


In [24]:
n=0
for k,v in tqdm_notebook(xref_chem.items()):
    for vv in list(v):
        if vv.startswith("UNII:"):
            xref = vv.replace("UNII:", "")
            s = unii_df.query("UNII == @xref").INCHIKEY
            if not s.empty:
                n+=1
                v.add("INCHIKEY:" + list(s)[0])

In [25]:
xref_inchi = {k:v for k,v in xref_chem.items() if any(vv.startswith("INCHIKEY:") for vv in v)}
xref_inchi = {k:[vv for vv in v if vv.startswith("INCHIKEY:")][0].replace("INCHIKEY:", "") for k,v in xref_inchi.items()}
print(len(xref_inchi))
list(xref_inchi.items())[:4]

9531


[('C0055257', 'GXGJIOMUZAGVEH-UHFFFAOYSA-N'),
 ('C0072534', 'ZPUCINDJVBIVPJ-XGUBFFRZSA-N'),
 ('C0600197', 'UKPBEPCQTDRZSE-UHFFFAOYSA-N'),
 ('C0764864', 'CHBOSHOWERDCMH-UHFFFAOYSA-N')]

In [26]:
url = "https://www.ebi.ac.uk/chembl/api/data/molecule?molecule_structures__standard_inchi_key__in={}&format=json&limit=100"
for chunk in tqdm(chunked(xref_inchi.items(), 100), total=len(xref_inchi)/100):
    chunk = dict(chunk)
    chunk = {v:k for k,v in chunk.items()}
    inchis = ",".join(chunk)
    mols = requests.get(url.format(inchis)).json()['molecules']
    for m in mols:
        chembl = m['molecule_chembl_id']
        inchi = m['molecule_structures']['standard_inchi_key']
        XREF[chunk[inchi]].add("CHEMBL:" + chembl)


  0%|          | 0/95.31 [00:00<?, ?it/s]
96it [16:36, 10.38s/it]                           


In [27]:
len({k:v for k,v in XREF.items() if any(vv.startswith("CHEMBL:") for vv in v)})

7718

In [28]:
with open("xrefs.shelve", 'wb') as f:
    pickle.dump(XREF, f)

In [29]:
## UBERON

In [30]:
!wget -N http://purl.obolibrary.org/obo/uberon.owl

--2018-07-02 16:38:41--  http://purl.obolibrary.org/obo/uberon.owl
Resolving purl.obolibrary.org (purl.obolibrary.org)... 52.3.123.63
Connecting to purl.obolibrary.org (purl.obolibrary.org)|52.3.123.63|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://ontologies.berkeleybop.org/uberon.owl [following]
--2018-07-02 16:38:41--  http://ontologies.berkeleybop.org/uberon.owl
Resolving ontologies.berkeleybop.org (ontologies.berkeleybop.org)... 54.230.86.136, 54.230.86.32, 54.230.86.160, ...
Connecting to ontologies.berkeleybop.org (ontologies.berkeleybop.org)|54.230.86.136|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67963495 (65M) [application/rdf+xml]
Saving to: ‘uberon.owl’

uberon.owl          100%[===================>]  64.81M  9.90MB/s    in 7.6s    

2018-07-02 16:38:49 (8.56 MB/s) - ‘uberon.owl’ saved [67963495/67963495]



In [31]:
s = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

SELECT * WHERE {
  ?item oboInOwl:hasDbXref ?xref
}
"""
with open("query.sparql", 'w') as f:
    f.write(s)

In [32]:
!robot query --input uberon.owl --query query.sparql uberon.csv

In [33]:
df = pd.read_csv("uberon.csv")
df = df[df.xref.str.startswith("UMLS:")]
df.xref = df.xref.str.replace("UMLS:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
6,UBERON:0006472,C1272528
73,UBERON:0001439,C0222661
96,UBERON:0001072,C0042458
153,UBERON:0001705,C0027342
183,UBERON:0002370,C0040113


In [34]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [35]:
XREF['C1272528']

{'UBERON:0006472'}

In [36]:
## DOID

In [37]:
!wget -N http://purl.obolibrary.org/obo/doid.owl

--2018-07-02 16:39:03--  http://purl.obolibrary.org/obo/doid.owl
Resolving purl.obolibrary.org (purl.obolibrary.org)... 52.3.123.63
Connecting to purl.obolibrary.org (purl.obolibrary.org)|52.3.123.63|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/doid.owl [following]
--2018-07-02 16:39:03--  https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/doid.owl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.24.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.24.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26462213 (25M) [text/plain]
Saving to: ‘doid.owl’

doid.owl            100%[===================>]  25.24M  17.5MB/s    in 1.4s    

Last-modified header missing -- time-stamps turned off.
2018-07-02 16:39:05 (17.5 MB/s) - ‘doid.owl’ saved [264622

In [38]:
!robot query --input doid.owl --query query.sparql doid.csv

In [39]:
df = pd.read_csv("doid.csv")
df.dropna(inplace=True)
df = df[df.xref.str.startswith("UMLS_CUI:")]
df.xref = df.xref.str.replace("UMLS_CUI:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
5,DOID:1943,C0263518
7,DOID:12960,C1510455
14,DOID:9455,C0029591
18,DOID:9123,C0936250
38,DOID:5591,C1367774


In [40]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [41]:
XREF['C0263518']

{'CCPSS:0019168',
 'DOID:1943',
 'HPO:HP:0025470',
 'ICD10:L65.0',
 'ICD10AM:L65.0',
 'ICD10CM:L65.0',
 'ICD9CM:704.02',
 'ICPC2ICD10ENG:MTHU025078',
 'ICPC2ICD10ENG:MTHU073653',
 'MDR:10043200',
 'MEDCIN:37106',
 'MTH:NOCODE',
 'NCI:C112200',
 'NCI_NICHD:C112200',
 'RCD:X509q',
 'SNM:M-58770',
 'SNMI:D0-53710',
 'SNOMEDCT_US:201147004',
 'SNOMEDCT_US:39479004'}

In [42]:
XREF['C0591520']

{'CHV:0000041179', 'MESH:D000068298', 'RCD:x02cs', 'RXNORM:151775'}

## proteins

In [44]:
# I did: cat MRSAT.RRF.a* > MRSAT.RRF
names = list("abcdefghijklmn")
iter_csv = pd.read_csv("MRSAT.RRF.gz", delimiter="|", names=names, index_col=None, chunksize=1000000)
chunks = []
umls_uniprot = dict()
for chunk in tqdm(iter_csv, total=67668372/1000000):
    chunk.fillna(method='ffill', inplace=True)
    chunk = chunk[chunk.i == "SWISS_PROT"]
    d = dict(zip(chunk.a, chunk.k))
    umls_uniprot.update(d)

 41%|████▏     | 28/67.668372 [00:54<01:17,  1.95s/it]/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
 46%|████▌     | 31/67.668372 [01:00<01:11,  1.94s/it]/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
71it [02:11,  1.85s/it]                               


In [45]:
len(umls_uniprot)

4493

In [46]:
for umls, uniprot in umls_uniprot.items():
    XREF[umls].add("UNIPROT:" + uniprot)

In [47]:
XREF['C0215993']

{'MESH:C081092', 'MTH:NOCODE', 'NCI:C127008', 'UNIPROT:Q04756'}

In [48]:
with open("xrefs.shelve", 'wb') as f:
    pickle.dump(XREF, f)

In [49]:
nodes['xrefs'] = nodes.index.map(lambda x: ";".join(XREF.get(x,list())))

In [50]:
nodes.head()

,LABEL,umls_type,umls_type_label,blm_type,xrefs
ID,,,,,
C0061133,gastrin releasing peptide (14-27),T116,"Amino Acid, Peptide, or Protein",protein,MESH:C041922
C1523610,"regulation of tube length, open tracheal system",T042,Organ or Tissue Function,biological_process_or_activity,GO:GO:0035159
C0312636,Antibody to hepatitis E virus,T116|T129,"Amino Acid, Peptide, or Protein|Immunologic Fa...",biological_entity,LNC:MTHU004056;SNMI:F-C2A90;MTH:NOCODE;SNOMEDC...
C0539817,cytochrome p30,T116|T126,"Amino Acid, Peptide, or Protein|Enzyme",protein,MESH:C106367
C0406240,Photosensitive atopic dermatitis,T047,Disease or Syndrome,disease_or_phenotypic_feature,RCD:X505U;SNOMEDCT_US:238548006;SNOMEDCT_US:23...


In [51]:
nodes.to_csv("nodes_xref.csv")